In [1]:
import sys, pathlib
sys.path.insert(0, str(pathlib.Path().resolve().parent.parent))

In [2]:
import triton
import triton.language as tl
import torch
import time
import pandas as pd
from conv_gemm.triton_kernels.int8.gemm_int8_kernel import gemm_int8_tc_kernel

In [3]:
def gemm_int8_tc(
    A_q: torch.Tensor,   
    B_q: torch.Tensor,  
    *,
    BLOCK_M: int = 64,
    BLOCK_N: int = 64,
    BLOCK_K: int = 32,
    num_warps: int = 4,
    num_stages: int = 2,
):
    if not A_q.is_contiguous():
        A_q = A_q.contiguous()
    if not B_q.is_contiguous():
        B_q = B_q.contiguous()

    M, K1 = A_q.shape
    K2, N = B_q.shape
    assert K1 == K2, f"K mismatch: {K1} vs {K2}"

    assert K1 % 4 == 0, f"K={K1} must be divisible by 4 for INT8 dot"
    assert BLOCK_K % 4 == 0, f"BLOCK_K={BLOCK_K} must be divisible by 4"

    C_i32 = torch.empty((M, N), dtype=torch.int32, device=A_q.device)

    a_m, a_k = A_q.stride()
    b_k, b_n = B_q.stride()
    c_m, c_n = C_i32.stride()

    grid = (
        triton.cdiv(M, BLOCK_M),
        triton.cdiv(N, BLOCK_N),
    )

    gemm_int8_tc_kernel[grid](
        A_q, B_q, C_i32,
        M, N, K1,
        a_m, a_k,
        b_k, b_n,
        c_m, c_n,
        BLOCK_M=BLOCK_M,
        BLOCK_N=BLOCK_N,
        BLOCK_K=BLOCK_K,
        num_warps=num_warps,
        num_stages=num_stages,
    )

    return C_i32

# title search

In [4]:
@torch.no_grad()
def bench_once_gemm_int8_vs_torch(
    M, K, N,
    BLOCK_M,
    BLOCK_N,
    BLOCK_K,
    num_warps,
    num_stages,
    iters=100,
    device="cuda",
):
    """
    INT8 Triton GEMM vs torch FP16 matmul
    """
    A_q = torch.randint(-128, 127, (M, K), device=device, dtype=torch.int8)
    B_q = torch.randint(-128, 127, (K, N), device=device, dtype=torch.int8)

    # масштаб для перевода int8 
    scale = 128.0
    s = 1.0 / scale

    # FP16 baseline на отмасштабированных данных
    A_f16 = (A_q.float() * s).half()
    B_f16 = (B_q.float() * s).half()

    # torch FP16 matmul
    def _call_torch():
        return A_f16 @ B_f16

    for _ in range(5):
        _ = _call_torch()
    torch.cuda.synchronize()

    t0 = time.perf_counter()
    for _ in range(iters):
        C_ref2 = _call_torch()
    torch.cuda.synchronize()
    t_torch = (time.perf_counter() - t0) / iters

    # Triton INT8 GEMM 
    def _call_triton():
        C_i32 = gemm_int8_tc(
            A_q, B_q,
            BLOCK_M=BLOCK_M,
            BLOCK_N=BLOCK_N,
            BLOCK_K=BLOCK_K,
            num_warps=num_warps,
            num_stages=num_stages,
        )
        return C_i32

    for _ in range(5):
        _ = _call_triton()
    torch.cuda.synchronize()

    t0 = time.perf_counter()
    for _ in range(iters):
        C_i32 = _call_triton()
    torch.cuda.synchronize()
    t_triton = (time.perf_counter() - t0) / iters

    # Error Triton INT32 vs Torch FP16
    C_tr_fp32  = C_i32.float() * (s * s)
    C_ref_fp32 = C_ref2.float()

    diff = (C_tr_fp32 - C_ref_fp32).abs()
    max_abs_err16 = diff.max().item()
    mean_abs_err16 = diff.mean().item()

    # bandwidth
    bytes_moved = A_q.numel() + B_q.numel()
    bytes_moved += C_i32.numel() * 4
    bytes_moved = float(bytes_moved)

    bw_triton = bytes_moved / t_triton / 1e9
    bw_torch  = bytes_moved / t_torch  / 1e9

    return {
        "M": M, "K": K, "N": N,
        "BLOCK_M": BLOCK_M,
        "BLOCK_N": BLOCK_N,
        "BLOCK_K": BLOCK_K,
        "num_warps": num_warps,
        "num_stages": num_stages,
        "t_triton_ms": t_triton * 1e3,
        "t_torch16_ms": t_torch * 1e3,
        "speed_vs_torch16": t_torch / t_triton,
        "bw_triton_GBs": bw_triton,
        "bw_torch16_GBs": bw_torch,
        "max_abs_err": max_abs_err16,
        "mean_abs_err": mean_abs_err16,
    }


In [5]:
@torch.no_grad()
def tune_gemm_int8_tiles_for_shape(
    M, K, N,
    blocks_M=(32, 64, 128),
    blocks_N=(32, 64, 128),
    blocks_K=(32, 64, 128),
    warps=(1, 2, 4, 8),
    stages=(2, 3,4),
    iters=200,
    device="cuda",
):
    records = []
    for BM in blocks_M:
        for BN in blocks_N:
            for BK in blocks_K:
                if (K % 4 != 0) or (BK % 4 != 0):
                    print(f"[SKIP] BM={BM}, BN={BN}, BK={BK}: K/BK not multiple of 4")
                    continue

                for W in warps:
                    for S in stages:
                        try:
                            rec = bench_once_gemm_int8_vs_torch(
                                M, K, N,
                                BLOCK_M=BM,
                                BLOCK_N=BN,
                                BLOCK_K=BK,
                                num_warps=W,
                                num_stages=S,
                                iters=iters,
                                device=device,
                            )
                        except RuntimeError as e:
                            print(f"[SKIP] BM={BM}, BN={BN}, BK={BK}, W={W}, S={S}: {e}")
                            continue

                        print(
                            f"BM={BM}, BN={BN}, BK={BK}, W={W}, S={S}: "
                            f"t_triton={rec['t_triton_ms']:.3f} ms, "
                            f"speed_vs_torch={rec['speed_vs_torch16']:.3f}x, "
                        )
                        records.append(rec)

    if not records:
        raise RuntimeError("No valid tile configs found for this GEMM shape")

    df = pd.DataFrame(records)
    return df

In [6]:
df_gemm_tiles = tune_gemm_int8_tiles_for_shape(
    M=4096, K=1024, N=1024,
    blocks_M=(32, 64, 128),
    blocks_N=(32, 64, 128),
    blocks_K=(32, 64, 128),
    warps=(2, 4, 8),
    stages=(2, 3),
    iters=200,
    device="cuda",
)

BM=32, BN=32, BK=32, W=2, S=2: t_triton=0.233 ms, speed_vs_torch=0.967x, 
BM=32, BN=32, BK=32, W=2, S=3: t_triton=0.234 ms, speed_vs_torch=0.967x, 
BM=32, BN=32, BK=32, W=4, S=2: t_triton=0.398 ms, speed_vs_torch=0.541x, 
BM=32, BN=32, BK=32, W=4, S=3: t_triton=0.392 ms, speed_vs_torch=0.544x, 
BM=32, BN=32, BK=32, W=8, S=2: t_triton=0.476 ms, speed_vs_torch=0.445x, 
BM=32, BN=32, BK=32, W=8, S=3: t_triton=0.460 ms, speed_vs_torch=0.461x, 
BM=32, BN=32, BK=64, W=2, S=2: t_triton=0.222 ms, speed_vs_torch=0.953x, 
BM=32, BN=32, BK=64, W=2, S=3: t_triton=0.216 ms, speed_vs_torch=1.012x, 
BM=32, BN=32, BK=64, W=4, S=2: t_triton=0.296 ms, speed_vs_torch=0.754x, 
BM=32, BN=32, BK=64, W=4, S=3: t_triton=0.333 ms, speed_vs_torch=0.718x, 
BM=32, BN=32, BK=64, W=8, S=2: t_triton=0.374 ms, speed_vs_torch=0.583x, 
BM=32, BN=32, BK=64, W=8, S=3: t_triton=0.364 ms, speed_vs_torch=0.585x, 
BM=32, BN=32, BK=128, W=2, S=2: t_triton=0.214 ms, speed_vs_torch=0.991x, 
BM=32, BN=32, BK=128, W=2, S=3: t_tri

In [7]:
df_gemm_tiles["shape_info"] = (
      "4096/1024/1024"

)

cols = [
    "shape_info",
    "BLOCK_M", "BLOCK_N", "BLOCK_K",
    "num_warps", "num_stages",
    "t_triton_ms", "t_torch16_ms",
    "speed_vs_torch16",
     "mean_abs_err"
]

df_gemm__filtered = df_gemm_tiles[cols].sort_values("speed_vs_torch16", ascending=False).head(5).reset_index(drop=True)
df_gemm__filtered

,shape_info,BLOCK_M,BLOCK_N,BLOCK_K,num_warps,num_stages,t_triton_ms,t_torch16_ms,speed_vs_torch16,mean_abs_err
0,4096/1024/1024,128,128,64,4,3,0.099355,0.216288,2.176926,0.001488
1,4096/1024/1024,64,128,64,4,3,0.104093,0.218960,2.103508,0.001488
2,4096/1024/1024,64,128,64,2,3,0.103166,0.213402,2.068533,0.001487
3,4096/1024/1024,128,128,64,4,2,0.108991,0.217836,1.998669,0.001489
4,4096/1024/1024,64,128,64,4,2,0.108386,0.213393,1.968820,0.001487


In [8]:
INT8_GEMM_BEST_BLOCK_M = 128
INT8_GEMM_BEST_BLOCK_N = 0
INT8_GEMM_BEST_BLOCK_K = 64
INT8_GEMM_BEST_WARPS   = 4
INT8_GEMM_BEST_STAGES  = 3

In [9]:

df_gemm__filtered.index = ["GEMM_INT8"] * len(df_gemm__filtered)
df_gemm__filtered

,shape_info,BLOCK_M,BLOCK_N,BLOCK_K,num_warps,num_stages,t_triton_ms,t_torch16_ms,speed_vs_torch16,mean_abs_err
GEMM_INT8,4096/1024/1024,128,128,64,4,3,0.099355,0.216288,2.176926,0.001488
GEMM_INT8,4096/1024/1024,64,128,64,4,3,0.104093,0.218960,2.103508,0.001488
GEMM_INT8,4096/1024/1024,64,128,64,2,3,0.103166,0.213402,2.068533,0.001487
GEMM_INT8,4096/1024/1024,128,128,64,4,2,0.108991,0.217836,1.998669,0.001489
GEMM_INT8,4096/1024/1024,64,128,64,4,2,0.108386,0.213393,1.968820,0.001487
